In [1]:
import json
import pika
import uuid
import threading 
import datetime
import time
import logging
import random
from flask import Flask, request
from pymessenger.bot import Bot
from pymessenger import Element, Button

In [2]:
def verify_fb_token(token_sent):
    print('paso 7')
    if token_sent == VERIFY_TOKEN:
        print('paso 7s')
        return request.args.get("hub.challenge")
    return 'Verificacion de token invalida'

In [3]:
def enviarNucleo(recipient_id, mensaje, output):
    global connection
    global channel
    global result
    global callback_queue
    global response
    global corr_id
        
    cont=0
    content=' '
    print('paso 8')
    if "postback" in mensaje[0]:
        print('paso 8s1')
        content=mensaje[0]["postback"]["title"]
    elif "message" in mensaje[0]:
        print('paso 8s1')
        content=mensaje[0]["message"]["text"]

    
    entidad={}
    estado = "none"
    tipo="none"
    opciones="none"            
        
    n=  {"payload": [{"robot":" ","recipient_id":" ","content":" ","state":" ", "blocktype":" ", "opciones":" ", "entitites":" "}]}
    data=json.dumps(n)
    decoded=json.loads(data)
    decoded["payload"][0]["robot"]="AAA1"
    decoded["payload"][0]["recipient_id"]=recipient_id
    decoded["payload"][0]["content"]=content
    decoded["payload"][0]["state"]=estado
    decoded["payload"][0]["blocktype"]=tipo
    decoded["payload"][0]["opciones"]=opciones
    decoded["payload"][0]["entities"]= entidad
    data_string=json.dumps(decoded)
    print(data_string)
    print('paso 9')
    corr_id = str(uuid.uuid4())
    channel.basic_publish(exchange='',
                                routing_key='chatbotV12',
                                properties=pika.BasicProperties(
                                        reply_to = callback_queue,
                                        correlation_id = corr_id,
                                        ),
                                body=str(data_string))
    print('paso 10')
            #while response is None:
             #                       connection.process_data_events()
            #data = json.loads(response)
            #entidad, resp, estado, tipo, opciones= conv(data,recipient_id)
            #entidad, resp, estado, tipo, opciones= conv(data)
            #print("LLego esto: ", data)
    #if(data["Respuesta"][0]["payload"][0]["state"]=="Salida"):                
     #   cont=-1
    #else:
     #   cont+=1

In [4]:
def send_message(recipient_id, response):
    global bot
    print('paso 19')
    #sends user the text message provided via input response parameter
    bot.send_text_message(recipient_id, response)
    return "success"

In [5]:
def test_button_message(recipient_id,content,options):
    global bot
    buttons = []
    print('paso 18')
    for opc in options:
        button = Button(title=opc, type='postback', payload=opc)
        buttons.append(button)
        
    text = content
    print('Text: ',text)
    print('Botones: ',buttons)
        
    result = bot.send_button_message(recipient_id, text, buttons)
    assert type(result) is dict
    assert result.get('message_id') is not None
    assert result.get('recipient_id') is not None

In [6]:
def conv(msj):
    #print(msj)
    print('paso 17')
    ans=' '
    contenido=msj["Respuesta"][0]["content"]
    estado=msj["Respuesta"][0]["payload"][0]["state"]
    entidad= msj["Respuesta"][0]["payload"][0]["entities"]
    tipo= msj["Respuesta"][0]["blockType"]
    recipient_id=msj['Respuesta'][0]['payload'][0]['recipient_id']
    print(str(contenido))
    #print(estado)
       
    if(tipo=="quickReply" or tipo=="quickReplyDinamico"):
        test_button_message(recipient_id, contenido, msj["Respuesta"][0]["options"])
        return entidad, ans, estado, tipo, msj["Respuesta"][0]["options"]
        
    elif(tipo=="input" or tipo=="inputDinamico"):
        send_message(recipient_id, contenido)
        return entidad, ans, estado, tipo, 'vacio'
        
    elif(tipo=="informativo" or tipo=="informativoDinamico"):
        send_message(recipient_id, contenido)
        return entidad, str('next'), estado, tipo, 'vacio'
    else:
        return entidad, str('hey'), estado, tipo, 'vacio'

In [7]:
class Nucleo_Cliente(threading.Thread):
    def __init__(self, nombreHilo, puerto, datos):        
        threading.Thread.__init__(self, name=nombreHilo, target=Nucleo_Cliente.run)
        print('paso 11')
        self.nombrehilo=nombreHilo
        #self.id_persona=id_persona
        #self.data=data
        
    def run(self):
        #self.consultar2(self.puerto, self datos)
        print('paso 12')
        response = self.call()
        print('paso 13')
        print(" [.] Got %r", response)              
        
    def on_response(self, ch, method, props, body):
        global response
        global corr_id
        print('paso on_response')
        if corr_id == props.correlation_id:
            response = body
            
    def call(self):
        global connection
        global channel
        global result
        global callback_queue
        global response
        
        cont=0
        #data_string = json.loads(body)
        #print(data_string)
        #print(n)
        #print(" [.] fib(%s)" % n)
        #while (not ("adios") in n):
        while(cont>=0):  
            print('paso 14')
            response = None
            #self.corr_id = str(uuid.uuid4())
            #channel.basic_publish(exchange='',
             #                          routing_key='chatbotV2',
              #                         properties=pika.BasicProperties(
               #                              reply_to = callback_queue,
                #                             correlation_id = self.corr_id,
                 #                            ),
                  #                     body=str(data_string))
            while response is None:
                                    connection.process_data_events()
            data = json.loads(response)
            print('Data-> ',data)
            print('paso 15')
            #entidad, resp, estado, tipo, opciones= conv(data,recipient_id)
            entidad, resp, estado, tipo, opciones= conv(data)
            print('paso 16')
            #print("LLego esto: ", data)
            #if(data["Respuesta"][0]["payload"][0]["state"]=="Salida"):
                
             #   cont=-1
            #else:
             #   cont+=1

In [8]:
#N_C = conversacion()
print('paso 1')
N_C=Nucleo_Cliente("Hilo_2", "3", " ")
response=None
corr_id=' '
connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
channel = connection.channel()
result = channel.queue_declare(exclusive=True)
callback_queue = result.method.queue
channel.basic_consume(N_C.on_response, no_ack=True, queue=callback_queue)
#N_C=Nucleo_Cliente("Hilo_2", "3", " ")
N_C.start()
#N_C.join()
print('paso 2')
app = Flask(__name__)
ACCESS_TOKEN = 'EAAIlqkJ5d4oBADlGkJrZBCzZCptOvFaXB7VT319dSTSXdVp5XljDxVPBQvEeRnVPC8sl51XMabDjgDBA11sZBuZCbTTAbPjb7JZArLjbpZCovZBMZBiUMnpRx7kswK8v1IdiBCaQzYKgmgn8chSbYsnZB25YQb15vV42MDZAa229AHL4WVxvLUIV01'
VERIFY_TOKEN = 'CadenaDeVerificacion3'
bot = Bot(ACCESS_TOKEN)
print('paso 3')
@app.route("/", methods=['GET', 'POST'])
def receive_message():
    print('paso 4')
    if request.method == 'GET':
        print('paso 4s1')
        token_sent = request.args.get("hub.verify_token")
        print('paso 4s2')
        return verify_fb_token(token_sent)
    else:
        output = request.get_json()
        print("output=  ",output)
        #mensaje=output['entry'][0]['messaging'][0]['message']['text']
        print('paso 4ss1')
        for event in output['entry']:
            print('paso 4s2')
            messaging = event['messaging']
            recipient_id=messaging[0]['sender']['id']
            enviarNucleo(recipient_id, messaging, output)
                         
    return "Message Processed"
        

if __name__ == "__main__":
    print('paso 6')
    app.run()

paso 1
paso 11
paso 12paso 2
paso 14

paso 3
paso 6
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Feb/2019 01:03:58] "POST / HTTP/1.1" 200 -


paso 4
output=   {'object': 'page', 'entry': [{'id': '573983246449458', 'time': 1550041437832, 'messaging': [{'sender': {'id': '1981139385336437'}, 'recipient': {'id': '573983246449458'}, 'timestamp': 1550041437541, 'message': {'mid': 'zazeEOgmMW0Hoqlg8CNLvk6-Bkdx6ilUH3ypsYVxIr-L5yhLMXr8mRUyo7yOKzdD0Qcguivfg0RzsL51Fomc8A', 'seq': 189677, 'text': 'Hola'}}]}]}
paso 4ss1
paso 4s2
paso 8
paso 8s1
{"payload": [{"robot": "AAA1", "recipient_id": "1981139385336437", "content": "Hola", "state": "none", "blocktype": "none", "opciones": "none", "entitites": " ", "entities": {}}]}
paso 9
paso 10
paso on_response
Data->  {'Respuesta': [{'content': 'Hola', 'next_id': 'InfoRobot', 'blockType': 'informativo', 'contentType': 'text', 'typingTime': '1', 'payload': [{'content': '', 'state': 'Saludo', 'recipient_id': '1981139385336437', 'entities': {}}]}]}
paso 15
paso 17
Hola
paso 19
paso 16
paso 14


[2019-02-13 01:03:59,194] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    retu

paso 4
output=   {'object': 'page', 'entry': [{'id': '573983246449458', 'time': 1550041438880, 'messaging': [{'sender': {'id': '1981139385336437'}, 'recipient': {'id': '573983246449458'}, 'timestamp': 1550041438868, 'delivery': {'mids': ['4YJ1-C0sqoWaTRYGvdcOG06-Bkdx6ilUH3ypsYVxIr9qWfWjHaw681YYbTUd3MNfefyIq-YFz07IKWVOBZn0BQ'], 'watermark': 1550041438541, 'seq': 0}}]}]}
paso 4ss1
paso 4s2
paso 8


...

paso 4
output=   {'object': 'page', 'entry': [{'id': '573983246449458', 'time': 1550041461742, 'messaging': [{'sender': {'id': '1981139385336437'}, 'recipient': {'id': '573983246449458'}, 'timestamp': 1550041461509, 'message': {'mid': 'HhLLM3FSWMF3k_sApJ71RU6-Bkdx6ilUH3ypsYVxIr9xO2GF4--H5AJgRoP-VqOOJbGxAXkGVTxd1NekNDL3XA', 'seq': 189697, 'text': 'no tengo'}}]}]}
paso 4ss1
paso 4s2
paso 8
paso 8s1
{"payload": [{"robot": "AAA1", "recipient_id": "1981139385336437", "content": "no tengo", "state": "none", "blocktype": "none", "opciones": "none", "entitites": " ", "entities": {}}]}
paso 9
paso 10
paso on_response
Data->  {'Respuesta': [{'content': 'Gracias vuelva pronto', 'next_id': 'Vacio', 'blockType': 'informativo', 'contentType': 'text', 'typingTime': '1', 'payload': [{'content': '', 'state': 'Salida', 'recipient_id': '1981139385336437', 'entities': {'VarQueja': 'no tengo'}}]}]}
paso 15
paso 17
Gracias vuelva pronto
paso 19
paso 16
paso 14


[2019-02-13 01:04:23,182] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    retu

paso 4
output=   {'object': 'page', 'entry': [{'id': '573983246449458', 'time': 1550041462834, 'messaging': [{'sender': {'id': '1981139385336437'}, 'recipient': {'id': '573983246449458'}, 'timestamp': 1550041462819, 'delivery': {'mids': ['Gp_otUW2HRJJhxt4Wv7jgE6-Bkdx6ilUH3ypsYVxIr9II2ujkQ7q0_DtheH_io4DfifQwcq56SLlR1SKHctqBw'], 'watermark': 1550041462431, 'seq': 0}}]}]}
paso 4ss1
paso 4s2
paso 8


[2019-02-13 01:04:24,069] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\eduar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    retu

paso 4
output=   {'object': 'page', 'entry': [{'id': '573983246449458', 'time': 1550041463706, 'messaging': [{'sender': {'id': '1981139385336437'}, 'recipient': {'id': '573983246449458'}, 'timestamp': 1550041463691, 'delivery': {'mids': ['4YJ1-C0sqoWaTRYGvdcOG06-Bkdx6ilUH3ypsYVxIr9qWfWjHaw681YYbTUd3MNfefyIq-YFz07IKWVOBZn0BQ', 'N9E7Zaqfv7I6uqNxQgT6eE6-Bkdx6ilUH3ypsYVxIr-CE3GShZtmkO89AzByC0oiB2-uupFL3Oz54k48CmMS5w', 'EHZq2Cy3zsxDvsVM9SyZbU6-Bkdx6ilUH3ypsYVxIr-Rl5bqNrtQNnS0x1D4fAgg2OYu_35fdG7k9GsGDZAoaw', '1oXezfHtlqcgehIoQeg1506-Bkdx6ilUH3ypsYVxIr8OfD59g59i2y5HDNi81kapyudKskwL1tOQRO9WziIDrw', 'Gp_otUW2HRJJhxt4Wv7jgE6-Bkdx6ilUH3ypsYVxIr9II2ujkQ7q0_DtheH_io4DfifQwcq56SLlR1SKHctqBw'], 'watermark': 1550041462431, 'seq': 0}}]}]}
paso 4ss1
paso 4s2
paso 8
